In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torchvision
from torchvision import transforms

from dataloading.nvidia import NvidiaDataset, NvidiaTrainDataset, NvidiaValidationDataset, NvidiaCropWide, Normalize
from network import PilotNet
from trainer import Trainer

%load_ext autoreload
%autoreload 2

In [2]:
root_path = Path("/media/romet/data2/datasets/rally-estonia/dataset")
tr = transforms.Compose([NvidiaCropWide(), Normalize()])
data_paths = [
    root_path / '2021-10-26-10-49-06_e2e_rec_ss20_elva',
    root_path / '2021-10-26-11-08-59_e2e_rec_ss20_elva_back']
nvidia_ds = NvidiaDataset(data_paths, tr)

# nvidia_ds.frames["turn_signal"] = nvidia_ds.frames["turn_signal"].astype(int)
# nvidia_ds.frames = nvidia_ds.frames[nvidia_ds.frames.turn_signal==1]

nvidia_loader = torch.utils.data.DataLoader(nvidia_ds, batch_size=64, shuffle=False,
                                         num_workers=32, pin_memory=True, persistent_workers=True)
len(nvidia_loader);

/media/romet/data2/datasets/rally-estonia/dataset/2021-10-26-10-49-06_e2e_rec_ss20_elva: 33045
/media/romet/data2/datasets/rally-estonia/dataset/2021-10-26-11-08-59_e2e_rec_ss20_elva_back: 33281


In [3]:
nvidia = PilotNet()
nvidia.load_state_dict(torch.load(f"models/20211103012304_autumn-v3/best.pt"))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
nvidia = nvidia.to(device);

In [4]:
trainer = Trainer()
nvidia_metrics = trainer.calculate_open_loop_metrics(nvidia, nvidia_loader, 30)
nvidia_metrics

  0%|          | 0/977 [00:00<?, ?it/s]

{'mae': 5.866918793786334,
 'rmse': 9.314956467847319,
 'max': 184.31200166227163,
 'whiteness': 75.57912,
 'expert_whiteness': 52.90005527856042}

In [5]:
from dataloading.ouster import OusterCrop, OusterNormalize, OusterDataset
tr = transforms.Compose([OusterCrop(), OusterNormalize()])
ouster_ds = OusterDataset(data_paths, transform=tr)

# ouster_ds.frames["turn_signal"] = ouster_ds.frames["turn_signal"].astype(int)
# ouster_ds.frames = ouster_ds.frames[ouster_ds.frames.turn_signal==1]

ouster_loader = torch.utils.data.DataLoader(ouster_ds, batch_size=64, shuffle=False,
                                         num_workers=32, pin_memory=True, persistent_workers=True)

/media/romet/data2/datasets/rally-estonia/dataset/2021-10-26-10-49-06_e2e_rec_ss20_elva: 11016
/media/romet/data2/datasets/rally-estonia/dataset/2021-10-26-11-08-59_e2e_rec_ss20_elva_back: 11094


In [6]:
ouster_model = PilotNet()
ouster_model.load_state_dict(torch.load(f"models/20211115214225_lidar-v3/best.pt"))
ouster_model = ouster_model.to(device);

In [7]:
ouster_metrics = trainer.calculate_open_loop_metrics(ouster_model, ouster_loader, 10)
ouster_metrics

  0%|          | 0/326 [00:00<?, ?it/s]

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warn

{'mae': 5.449213296351707,
 'rmse': 8.887391789811598,
 'max': 293.40379198589966,
 'whiteness': 44.506634,
 'expert_whiteness': 32.727281029741896}

In [8]:
metrics_df = pd.DataFrame(columns=["model", "mae", "rmse", "max", "whiteness", "expert_whiteness"])

nvidia_metrics["model"] = "nvidia"
metrics_df = metrics_df.append(nvidia_metrics, ignore_index=True)

ouster_metrics["model"] = "ouster"
metrics_df = metrics_df.append(ouster_metrics, ignore_index=True)

metrics_df

,model,mae,rmse,max,whiteness,expert_whiteness
0,nvidia,5.866919,9.314956,184.312002,75.579117,52.900055
1,ouster,5.449213,8.887392,293.403792,44.506634,32.727281
